In [17]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
spark = SparkSession.builder \
    .appName("PySpark Azure Cosmos DB Example") \
    .config("spark.jars.packages", "com.microsoft.azure:azure-cosmosdb-spark_2.12:3.6.0") \
    .config("spark.cosmos.accountEndpoint", "https://ney.documents.azure.com:xxx/") \
    .config("spark.cosmos.accountKey", "000000000000000000000000000000000000") \
    .config("spark.cosmos.database", "xxx") \
    .config("spark.cosmos.container", "xxx1") \
    .getOrCreate()



    
    

In [18]:
class blob_store_relatorio_repository:

    def __init__(self):
        self.storage_account_name = "dadoswalney"
        self.storage_account_access_key = "413123"
        self.container_name = "neydados"
        
    def get_blob_dados_vendas(self):
        spark.conf.set("fs.azure.account.key." + self.storage_account_name + ".blob.core.windows.net", self.storage_account_access_key)
        blob_file_path = f"wasbs://{self.container_name}@{self.storage_account_name}.blob.core.windows.net/tbvendas.csv"

        return spark.read.csv(blob_file_path, header=True)

    def get_blob_dados_funcionario(self):
        spark.conf.set("fs.azure.account.key." + self.storage_account_name + ".blob.core.windows.net", self.storage_account_access_key)
        blob_file_path = f"wasbs://{self.container_name}@{self.storage_account_name}.blob.core.windows.net/tbfuncionario.csv"

        return spark.read.csv(blob_file_path, header=True)

In [33]:
x=blob_store_relatorio_repository()
x.get_blob_dados_funcionario()

DataFrame[id_funcionario: string, nome: string, cargo: string, departamento: string, data_admissao: string, salario: string]

In [19]:
class  blob_store_relatorio_controller_vendas:
    def  __init__(self):
        self.repositorio= blob_store_relatorio_repository()
    
    def obter_dados_vendas(self):
        return self.repositorio.get_blob_dados_vendas()
    
    

class  blob_store_relatorio_controller_funcionario:
    def  __init__(self):
        self.repositorio= blob_store_relatorio_repository()
    
    def obter_dados_funcionario(self):
        return self.repositorio.get_blob_dados_funcionario()
    
        
    

In [77]:
xx=blob_store_relatorio_controller_vendas()
xx.obter_dados_vendas()

DataFrame[id_venda: string,  id_funcionario: string,  data_venda: string,  produto: string,  quantidade: string,  preco_unitario: string,  total_venda: string]

In [122]:
df_vendas

DataFrame[id_venda: string,  id_funcionario: string,  data_venda: string,  produto: string,  quantidade: string,  preco_unitario: string,  total_venda: string]

In [20]:
#instanciar objetos vendas e funcionario

df_vendas= blob_store_relatorio_controller_vendas().obter_dados_vendas()
df_funcionario = blob_store_relatorio_controller_funcionario().obter_dados_funcionario()

In [133]:
df_funcionario

DataFrame[id_funcionario: string, nome: string, cargo: string, departamento: string, data_admissao: string, salario: string]

In [150]:
df_vendas.show()

+--------+---------------+-----------+-----------+-----------+---------------+------------+
|id_venda| id_funcionario| data_venda|    produto| quantidade| preco_unitario| total_venda|
+--------+---------------+-----------+-----------+-----------+---------------+------------+
|       1|              1| 2024-06-01|      Mouse|          2|          25.00|       50.00|
|       2|              2| 2024-06-02|    Teclado|          1|          40.00|       40.00|
|       3|              1| 2024-06-03|    Monitor|          1|         150.00|      150.00|
|       4|              3| 2024-06-04|    Headset|          3|          30.00|       90.00|
|       5|              2| 2024-06-05| Impressora|          1|         200.00|      200.00|
|       6|              1| 2024-06-06|    Teclado|          2|          40.00|       80.00|
|       7|              2| 2024-06-07|    Monitor|          1|         150.00|      150.00|
|       8|              3| 2024-06-08|      Mouse|          3|          25.00|  

In [130]:
type(df_funcionario)

NoneType

In [21]:
# tratamento 1 : obter os campos de destino de tabela
from pyspark.sql.functions import col
df_vendas=df_vendas.select('id_venda',col(' id_funcionario').alias('id_fun'),' preco_unitario',' produto',' data_venda' )

In [22]:
df_funcionario=df_funcionario.select('id_funcionario','nome','cargo','salario')

In [173]:
df_vendas.show()

+--------+------+---------------+-----------+-----------+
|id_venda|id_fun| preco_unitario|    produto| data_venda|
+--------+------+---------------+-----------+-----------+
|       1|     1|          25.00|      Mouse| 2024-06-01|
|       2|     2|          40.00|    Teclado| 2024-06-02|
|       3|     1|         150.00|    Monitor| 2024-06-03|
|       4|     3|          30.00|    Headset| 2024-06-04|
|       5|     2|         200.00| Impressora| 2024-06-05|
|       6|     1|          40.00|    Teclado| 2024-06-06|
|       7|     2|         150.00|    Monitor| 2024-06-07|
|       8|     3|          25.00|      Mouse| 2024-06-08|
|       9|     1|          40.00|    Teclado| 2024-06-09|
|      10|     2|         150.00|    Monitor| 2024-06-10|
+--------+------+---------------+-----------+-----------+



In [174]:
df_funcionario.show()

+--------------+------+---------------+-------+
|id_funcionario|  nome|          cargo|salario|
+--------------+------+---------------+-------+
|             1|  João|Sales Associate|2500.00|
|             2| Maria|  Sales Manager|3500.00|
|             3|Carlos|     Accountant|3000.00|
+--------------+------+---------------+-------+



In [175]:
df_vendas.printSchema()

root
 |-- id_venda: string (nullable = true)
 |-- id_fun: string (nullable = true)
 |--  preco_unitario: string (nullable = true)
 |--  produto: string (nullable = true)
 |--  data_venda: string (nullable = true)



In [176]:
df_funcionario.printSchema()

root
 |-- id_funcionario: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: string (nullable = true)



In [23]:
#tratamento de dados

"""dataframe funcionario"""
df_funcionario = df_funcionario.withColumn("salario", col("salario").cast('float'))
df_funcionario = df_funcionario.withColumn("id_funcionario", col("id_funcionario").cast("int"))


"""dataframe vendas"""
#eliminar o espaco do  atributo preco_unitario
df_vendas = df_vendas.withColumn("preco_unitario", col(" preco_unitario").cast("float")).drop(" preco_unitario")
#eliminar o espaco do  atributo data_venda
df_vendas = df_vendas.withColumn("data_venda", col(" data_venda").cast('date')).drop(col(" data_venda"))
#renomer o atributo produto para o nome
df_vendas = df_vendas.withColumnRenamed(" produto", "nome_produto")


In [24]:

df_funcionario.printSchema()

root
 |-- id_funcionario: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: float (nullable = true)



In [197]:
df_vendas.printSchema()

root
 |-- id_venda: string (nullable = true)
 |-- id_fun: string (nullable = true)
 |-- nome_produto: string (nullable = true)
 |-- preco_unitario: float (nullable = true)
 |-- data_venda: date (nullable = true)



In [25]:
"""consolidacao de dados"""

df_r_func_vend= df_funcionario.join(df_vendas,df_vendas.id_fun==df_funcionario.id_funcionario,'left')

In [33]:
df_r_func_vend.printSchema()

root
 |-- id_funcionario: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: float (nullable = true)
 |-- id_venda: string (nullable = true)
 |-- id_fun: string (nullable = true)
 |-- nome_produto: string (nullable = true)
 |-- preco_unitario: float (nullable = true)
 |-- data_venda: date (nullable = true)



In [26]:


df_r_func_vend = (df_r_func_vend.withColumn('colun', 
                                            row_number().over(Window.partitionBy('id_fun')
                                                              .orderBy(col('data_venda').desc())))
).filter(col('colun')==1).select('cargo','data_venda','id_fun','nome','nome_produto')



In [13]:
df_r_func_vend.show(4)

+---------------+----------+------+------+------------+
|          cargo|data_venda|id_fun|  nome|nome_produto|
+---------------+----------+------+------+------------+
|Sales Associate|2024-06-09|     1|  João|     Teclado|
|  Sales Manager|2024-06-10|     2| Maria|     Monitor|
|     Accountant|2024-06-08|     3|Carlos|       Mouse|
+---------------+----------+------+------+------------+



In [14]:
df_r_func_vend

DataFrame[cargo: string, data_venda: date, id_fun: string, nome: string, nome_produto: string]

In [ ]:

df_r_func_vend.write.format("cosmos.oltp").mode("append").save()
